In [ ]:
# develop-hamming-distance-permutation.ipynb
#
# Bryan Daniels, Enrico Borriello
# 2023/9/19
#

In [4]:
import attattach as at
import numpy as np

In [5]:
def label_to_state (label, digits):
    return np.array(list(map(int,list(format(label,'0'+str(digits)+'b')))))

def state_to_label (state):
    return int(''.join(map(str,state)),2)

In [6]:
def hamming_distance(label1,label2,n):
    return np.sum(abs(label_to_state(label1,n) - label_to_state(label2,n)))

In [7]:
hamming_distance(1,3,n)

NameError: name 'n' is not defined

In [13]:
hamming_distance(1,5,n)

1

In [19]:
hamming_distance(21,21,n)

0

In [16]:
label_to_state(21,n)

array([1, 0, 1, 0, 1])

In [24]:
state_to_label([1,0,1,0,0])

20

In [7]:
def labels_permutation_close(transitions):
    """
    randomly reassign state labels,
    with bias toward states that are closer in
    Hamming distance
    """
    # Generate a random permutation of the labels
    s = len(transitions)
    p = list(range(s))
    random.shuffle(p)
    # Create a new list of edges with the updated labels
    new_transitions = [(p[i], p[j]) for (i, j) in transitions]
    sorted_transitions = sorted(new_transitions, key=lambda x: x[0])
    return sorted_transitions

In [8]:
def generate_landscape(num_nodes,landscape_structure,close=True):
    """
    Sample landscape structure: [[3,.25],[1,.50],[1,.05],[2,.20]]
    This corresponds to 4 attractors, of lengths 3, 1, 1, and 2,
    with relative basins sizes equal to 25%, 50%, 5%, and 20%
    """
    
    s = 2**num_nodes # total number of states in the attractor landscape

    # Read the structure of the attractor landscape
    lengths = [B[0] for B in landscape_structure]
    rel_sizes = [B[1] for B in landscape_structure]
    sizes = [int(rel_size*s) for rel_size in rel_sizes] # the last one might be wrong
    sizes[-1] = s-(sum(sizes)-sizes[-1]) # this fixes it
    # attractor states in each basin:
    num_att_states = [ landscape_structure[i][0] for i in range(len(landscape_structure)) ]
    
    # CONDITION 1:
    # 'The sum of the relative basin sizes needs to be 1'
    c1 = np.allclose(sum(rel_sizes),1.) 

    # CONDITION 2:
    # All the basins have at least size 1 
    # (For small n and small relative size of a basin, the product might result in zero states)
    c2 = np.prod([sizes[i] > 0 for i in range(len(sizes))])

    # CONDITION3:
    # There are at least as many states as attractor states
    c3 = np.sum(num_att_states) <= s

    # CONDITION4:
    # There are at least as many states as attractor states **in each individual basin**
    c4 = np.prod([ num_att_states[i] <= sizes[i] for i in range(len(landscape_structure)) ])

    # If all conditions are satisfied, proceed:
    if c1*c2*c3*c4:
    
        # generate the individual basins
        t = []
        for i in range(len(landscape_structure)):
            t.append(transitions(lengths[i], sizes[i]))

        # join them with the sequential relabeling 
        all_t = []
        for i in range(len(t)):
            all_t = join_transitions(all_t,t[i])
    
        if close:
            return labels_permutation_close(all_t)
        else:
            return labels_permutation(all_t)

    else:
        if not c1:
            print('ERROR: The sum of the relative basin sizes is not 1.')
        if not c2:
            print('ERROR: At least one basin has size 0.')
            print('       (relative size is too small for your n).')
        if not c3:
            print('ERROR: There are more attractor states than total states.')
        if not c4:
            print('ERROR: There are more attractor states than total states in at least one basin.')
        return None


# 2023/9/26 Enrico's idea: subgraph isomorphism (MONOmorphism!)

If we create a directed graph GH that connects all states within a given Hamming distance threshold, and a second directed graph GT with the desired transition structure, then there exists a permutation of the labels of GT such that transitions are always below the Hamming distance threshold if GT is a subgraph of GH.  And if that isomorphism is found, we can use the corresponding permutation for the `labels_permutation_close` function.

In [8]:
import networkx as nx
from networkx.algorithms import isomorphism

In [133]:
at

<module 'attattach' from '/Users/bdaniel6/packages/AttAttach/AttAttach/attattach.py'>

In [11]:
landscape_structure = [[1,.25],[1,.50],[1,.05],[1,.20]]
n = 5
ls = at.generate_landscape(n,landscape_structure,close=False)

In [70]:
GT_directed_edges = ls

In [69]:
GH_directed_edges = generate_edges_maximum_H(n,hamming_threshold)

In [71]:
GH_reversed_edges = [ (i,j) for (j,i) in GH_directed_edges ]

In [76]:
len(GH_directed_edges)

528

In [75]:
len(GH_directed_edges + GH_reversed_edges)

1056

In [80]:
net_new = nx.DiGraph(GH_directed_edges + GH_reversed_edges)

In [12]:
def flip_bits(label,bit_indices,n):
    """
    Flip bits of state corresponding to label at indices given by bit_indices
    """
    state = label_to_state(label,n)
    for i in bit_indices:
        if state[i] == 0:
            state[i] = 1
        else:
            state[i] = 0
    return state_to_label(state)

In [45]:
flip_bits(10,[0,1],10)

778

In [13]:
# old version that only works for H=1
# def hamming_network(n,threshold):
#     """
#     Network connecting all states with Hamming distance less than or equal to threshold.
#     """
#     edgelist = []
#     for state in range(2**n):
#         for h in range(threshold):
#             # TO DO: FIX THE FOLLOWING LINES: CURRENTLY JUST INCLUDES STATES AT HAMMING DISTANCE 1
#             for i in range(n):
#                 bit_indices = [i]
#                 edgelist.append((state,flip_bits(state,bit_indices,n)))
#     return nx.DiGraph(edgelist)

In [52]:
from itertools import combinations

def hamming_distance(v1, v2):
    return sum(x != y for x, y in zip(v1, v2))

def generate_edges_fixed_H(n, H):
    """
    (Doesn't work with H=0)
    """
    assert(H>0)
    vertices = [(i, format(i, f'0{n}b')) for i in range(2**n)]
    edges = []
    for v1, v2 in combinations(vertices, 2):
        if hamming_distance(v1[1], v2[1]) == H:
            edges.append((v1[0], v2[0]))
    return edges

def generate_edges_maximum_H(n, Hmax):
    edges = [ (i,i) for i in range(2**n) ]
    for H in range(1,Hmax+1):
        edges = edges + generate_edges_fixed_H(n,H)
    return edges

In [15]:
GT

In [20]:
n

5

In [23]:
2**5

32

In [24]:
len(GH.edges)

496

In [25]:
32*31/2

496.0

In [132]:
# example of searching for isomorphisms using NetworkX:
# (see https://networkx.org/documentation/stable/reference/algorithms/isomorphism.vf2.html )
hamming_threshold = 5
GT = nx.DiGraph(ls)
GH = nx.DiGraph(generate_edges_maximum_H(n,hamming_threshold))
GH_new = nx.DiGraph(list(GH.edges()) + list(GH.reverse().edges()))

DiGM = isomorphism.DiGraphMatcher(GH_new, GT)
if DiGM.subgraph_is_monomorphic():
    print(DiGM.mapping)
else:
    print("no isomorphism found")

{0: 0, 1: 20, 2: 6, 3: 16, 4: 17, 5: 18, 6: 22, 7: 28, 8: 1, 9: 7, 10: 3, 11: 4, 12: 5, 13: 12, 14: 9, 15: 10, 16: 13, 17: 14, 18: 23, 19: 24, 20: 25, 21: 26, 22: 27, 23: 30, 24: 2, 25: 19, 26: 21, 27: 29, 28: 8, 29: 11, 30: 31, 31: 15}


In [86]:
DiGM = isomorphism.DiGraphMatcher(GT,net_new)
DiGM.subgraph_is_isomorphic()

In [102]:
net_zero = nx.DiGraph([[0,0],[0,2]])
net_one = nx.DiGraph([[0,0],[0,1],[1,0],[1,1],[1,2],[2,1],[2,2],[0,2],[2,0]])

In [103]:
DiGM = isomorphism.DiGraphMatcher(net_one,net_zero)
DiGM.subgraph_is_isomorphic()

False

In [127]:
net_a = nx.DiGraph([[0,1],[1,2],[2,0],[1,0]])
net_b = nx.DiGraph([[1,2],[2,0],[0,1],[0,2],[3,0]])

In [128]:
DiGM = isomorphism.DiGraphMatcher(net_b,net_a)
DiGM.subgraph_is_isomorphic()

True

In [129]:
net_a = nx.DiGraph([[0,1],[1,2],[2,0],[1,0]])
net_b = nx.DiGraph([[1,2],[2,0],[0,1],[0,2],[3,0],[1,0]])

In [130]:
DiGM = isomorphism.DiGraphMatcher(net_b,net_a)
DiGM.subgraph_is_isomorphic()

False

In [131]:
DiGM.subgraph_is_monomorphic()

True